# COVID19 International V2

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-setup" data-toc-modified-id="Python-setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python setup</a></span></li><li><span><a href="#Get-the-raw-data-and-adjust-for-anomalies" data-toc-modified-id="Get-the-raw-data-and-adjust-for-anomalies-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get the raw data and adjust for anomalies</a></span><ul class="toc-item"><li><span><a href="#Nation-naming-stuff" data-toc-modified-id="Nation-naming-stuff-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Nation naming stuff</a></span></li><li><span><a href="#COVID-data-retrieval-and-adjustment-for-outliers" data-toc-modified-id="COVID-data-retrieval-and-adjustment-for-outliers-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>COVID data retrieval and adjustment for outliers</a></span></li></ul></li><li><span><a href="#Plot-weekly-new-case/deaths-data-from-January-2020" data-toc-modified-id="Plot-weekly-new-case/deaths-data-from-January-2020-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot weekly new-case/deaths data from January 2020</a></span></li><li><span><a href="#Plot-new-vs-cumulative-(all-and-last-3-months)" data-toc-modified-id="Plot-new-vs-cumulative-(all-and-last-3-months)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot new-vs-cumulative (all and last 3 months)</a></span></li><li><span><a href="#International-comparisons---maps/leader-boards/swarms" data-toc-modified-id="International-comparisons---maps/leader-boards/swarms-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>International comparisons - maps/leader-boards/swarms</a></span></li><li><span><a href="#Semi-log-comparison-plots-of-cumulative" data-toc-modified-id="Semi-log-comparison-plots-of-cumulative-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Semi-log comparison plots of cumulative</a></span></li><li><span><a href="#Comparison-plots-of-daily-new-per-capita" data-toc-modified-id="Comparison-plots-of-daily-new-per-capita-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Comparison plots of daily new per capita</a></span></li><li><span><a href="#Growth" data-toc-modified-id="Growth-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Growth</a></span></li><li><span><a href="#The-End" data-toc-modified-id="The-End-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>The End</a></span></li></ul></div>

## Python setup

In [1]:
# system imports
import sys
from pathlib import Path

#analytic imports
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.dates as mdates
import matplotlib.units as munits
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import geopandas as gpd
import iso3166

# COVID19 specific imports
sys.path.append(r'../bin')
import datagrabber as dg
import plotstuff as ps

# directory
CHART_DIRECTORY = '../charts'
Path(CHART_DIRECTORY).mkdir(parents=True, exist_ok=True)
CHART_DIRECTORY += '/'
I_PREFIX = '!I-'

# display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')
register_matplotlib_converters()

## Get the raw data and adjust for anomalies

### Nation naming stuff

In [2]:
# make country names ISO compliant
iso_name_map = {
    # from              # to
    "Bolivia":          "Bolivia, Plurinational State of",
    "Bonaire, Saint Eustatius And Saba": "Bonaire, Sint Eustatius and Saba",
    "British Virgin Islands": "Virgin Islands, British",
    "Brunei":           "Brunei Darussalam",
    "Cape Verde":       "Cabo Verde",
    "Cote D'Ivoire":    "Côte d'Ivoire",
    "Cote d'Ivoire":    "Côte d'Ivoire",
    "Curaã§Ao":         "Curaçao",
    "Democratic Republic Of Congo": "Congo, Democratic Republic of the",
    "Democratic Republic of Congo": "Congo, Democratic Republic of the",
    "Guinea Bissau":    "Guinea-Bissau",
    "Vatican":          "Holy See",
    "Iran":             "Iran, Islamic Republic of",
    "Laos":             "Lao People's Democratic Republic",
    "Moldova":          "Moldova, Republic of",
    "Russia":           "Russian Federation",
    "Sint Maarten":     "Sint Maarten (Dutch part)",
    "South Korea":      "Korea, Republic of",
    "Syria":            "Syrian Arab Republic",
    "Tanzania":         "Tanzania, United Republic of",
    "Timor":            "Timor-Leste",
    "Timor Leste":      "Timor-Leste",
    "United Kingdom":   "United Kingdom of Great Britain and Northern Ireland",
    "United Republic Of Tanzania": "Tanzania, United Republic of",
    "United States":    "United States of America",
    "United States Virgin Islands":   "Virgin Islands, U.S.",
    "Venezuela":        "Venezuela, Bolivarian Republic of",
    "Vietnam":          "Viet Nam",
}

must_name_map = {
    # these names from the EU source are ugly and we will change anyway
    "Cote D'Ivoire":    "Côte d'Ivoire",
    "Curaã§Ao":         "Curaçao",
}

In [3]:
def name_map(object_, map_):
    """Rename the index of a Series or the columns of a DataFrame
       The renaming is done in place.
       Parameters
       - object_ - the pandas Series or DataFrame
       - map_ - a dictionary of old to new mappings 
       Returns: None """

    fixing = {}
    if isinstance(object_, pd.Series):
        for name in map_:
            if name in object_.index:
                fixing[name] = map_[name]
        object_.rename(index=fixing, inplace=True)
    else:
        for name in map_:
            if name in object_.columns:
                fixing[name] = map_[name]
        object_.rename(columns=fixing, inplace=True)
    
    return None

In [4]:
def get_national_code(nation):
    if nation in iso_name_map.keys():
        nation = iso_name_map[nation]
    if nation == 'European Union':
        return 'EU'
    code = iso3166.countries.get(nation).alpha2
    return code

### COVID data retrieval and adjustment for outliers

In [5]:
# retrieve raw
raw_cases, raw_deaths, population, source = dg.get_OWID_data()

# apply compulsory name changes
name_map(population, must_name_map)
name_map(raw_cases, must_name_map)
name_map(raw_deaths, must_name_map)

https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv
We have the data


In [6]:
# identify the source
source

'OWID 2020-12-23'

In [7]:
# check for missing data in the last row
for mode, data in zip(['cases', 'death'],
                      [raw_cases, raw_deaths]):
    missing = data.iloc[-1]
    missing = missing[missing.isna()]
    if len(missing):
        print(f'Missing {mode}')
        display(data[missing.index].iloc[-7])

In [8]:
# make adjustments to the data
CASES = 0
DEATHS = 1
modes = {
    'Cases': CASES,
    'Deaths': DEATHS,
}
raw_cum_data = [raw_cases, raw_deaths]
raw_daily_data = [None, None]
adj_daily_data = [None, None]
adj_cum_data = [None, None]

data_quality = [None, None]

for mode, index in modes.items():
    
    name_map(raw_cum_data[index], must_name_map)
    
    # make missing data zero
    raw_cum_data[index] = raw_cum_data[index].fillna(0)
    
    # adjust data for anomalies
    (raw_daily_data[index], 
        adj_daily_data[index], 
        adj_cum_data[index]) = ps.dataframe_correction(
                                    raw_cum_data[index])
    
    # identify whether the adjustment for anomalies 
    # changed the data
    data_quality[index] = pd.Series(None, 
                            index=raw_cum_data[index].columns,
                            dtype='str')
    for col in raw_cum_data[index].columns:
        if (raw_daily_data[index][col] == 
            adj_daily_data[index][col]).all():
            data_quality[index][col] = (f'Source: {source}, '
                                         'original data')
        else:
            data_quality[index][col] = (f'Source: {source}, '
                            'data adjusted for extreme outliers')

Spikes in Andorra
date   2020-06-02
spike   79.000000
mean     0.714286
zeros   10.000000
Negatives in Antigua and Barbuda
date
2020-07-03   -1.0
Name: Antigua and Barbuda, dtype: float64
Data too sparse in Antigua and Barbuda (max_consecutive=2)
Data too sparse in Barbados (max_consecutive=10)
Spikes in Belize
date    2020-12-03
spike  1382.000000
mean    142.857143
zeros     2.000000
Negatives in Benin
date
2020-05-19   -209.0
Name: Benin, dtype: float64
Data too sparse in Benin (max_consecutive=10)
Data too sparse in Bhutan (max_consecutive=9)
Data too sparse in Botswana (max_consecutive=2)
Spikes in Burkina Faso
date   2020-05-06  2020-06-02  2020-09-13
spike   41.000000   34.000000       193.0
mean     6.714286    1.785714        17.5
zeros    0.000000    6.000000         0.0
Data too sparse in Burundi (max_consecutive=10)
Data too sparse in Cambodia (max_consecutive=14)
Data too sparse in Cameroon (max_consecutive=7)
Spikes in Cape Verde
date   2020-04-15
spike        45.0
mean  

Negatives in Austria
date
2020-07-21   -1.0
2020-10-11   -1.0
Name: Austria, dtype: float64
Data too sparse in Bahamas (max_consecutive=9)
Data too sparse in Barbados (max_consecutive=3)
Negatives in Belgium
date
2020-08-26   -117.0
Name: Belgium, dtype: float64
Data too sparse in Benin (max_consecutive=3)
Spikes in Bolivia
date    2020-09-07
spike  1656.000000
mean     55.142857
zeros     1.000000
Data too sparse in Botswana (max_consecutive=1)
Data too sparse in Brunei (max_consecutive=1)
Data too sparse in Burkina Faso (max_consecutive=5)
Data too sparse in Burundi (max_consecutive=1)
Data too sparse in Cameroon (max_consecutive=5)
Spikes in Canada
date   2020-10-02
spike   98.000000
mean    17.357143
zeros    0.000000
Data too sparse in Cape Verde (max_consecutive=5)
Data too sparse in Central African Republic (max_consecutive=5)
Data too sparse in Chad (max_consecutive=6)
Spikes in Chile
date   2020-06-08   2020-07-17
spike  627.000000  1057.000000
mean   120.071429    83.928571
z

## Plot weekly new-case/deaths data from January 2020

In [9]:
def plot_weekly(daily, mode, data_quality, dfrom="2020-01-21"):
    """Plot weekly bar charts for daily new cases and deaths
        Function paramters:
        - daily is a DataFrame of daily timeseries data
        - mode is one of 'cases' or 'deaths' 
        - data_quality is a Series of strings,
            used for the left footnote on charts
        - dfrom is a date string to display from
        Returns: weekly data in a DataFrame """
    
    DISPLAY_FROM = pd.Timestamp(dfrom)
    
    # find the day that the week ends - last day of dataframe
    LAST_DAY = daily.index[-1]
    RULE = {
        0: 'W-MON',
        1: 'W-TUE',
        2: 'W-WED',
        3: 'W-THU',
        4: 'W-FRI',
        5: 'W-SAT',
        6: 'W-SUN',
    }[LAST_DAY.dayofweek]

    # convert the data to weekly
    returnable = weekly = daily.resample(rule=RULE, 
                                         closed='right').sum()
    total = weekly.sum()
    
    # adjust data and dates for plotting
    # we move the data by half a week becuase 
    # we want the bars to be centred
    weekly = weekly[weekly.index > DISPLAY_FROM]
    weekly.index = weekly.index - pd.Timedelta(3.5, unit='d')
    
    for name in daily.columns:
    
        # avoid plotting an empty plot
        if total[name] == 0: continue
    
        # plot the data
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.bar(weekly.index, weekly[name], width=5, color='#dd0000', )
        ax.margins(0.01)
            
        # This makes the dates for xticklabels look a little nicer
        locator = mdates.AutoDateLocator(minticks=4, maxticks=13)
        formatter = mdates.ConciseDateFormatter(locator)
        ax.xaxis.set_major_locator(locator)
        ax.xaxis.set_major_formatter(formatter)

        # adjust y-limits to be prettier, 
        # assume ylim[0] is zero
        # this adjustment should not be needed, but it is
        ylim = ax.get_ylim()
        ylim = ylim[0], ylim[1] * 1.025
        if ylim[0] != 0:
            # this should not happen - ever.
            print(f'Warning: ylim[0] is {ylim[0]} for {name}')
        
        # an ugly kludge for putting commas in the ylabels
        def small(x): return np.round(x, 2)
        def big(x): return int(x)
        functor = small if weekly[name].max() <= 4 else big
        ax.get_yaxis().set_major_formatter(
            mpl.ticker.FuncFormatter(
                lambda x, p: format(functor(x), ',')))
            
        ps.finalise_plot(ax, 
                         title=f'COVID-19 {mode.title()}: {name}',
                         xlabel=None,
                         ylabel=f'New {mode.lower()}/week ending '
                                f'{RULE[-3:].title()}',
                         ylim=ylim,
                         rfooter=data_quality[index][name],
                         lfooter=f'Total {mode.lower()}: '
                                 f'{int(total[name]):,}',
                         save_as=f'{CHART_DIRECTORY}'
                                 f'{name}-{mode.lower()}'
                                  '-new-weekly.png'
                         )
        
    return returnable

In [10]:
if True: # switch this output on/off
    adj_weekly_data = [None, None]
    for (mode, index) in modes.items():
        daily = adj_daily_data[index].copy()
        adj_weekly_data[index] = plot_weekly(daily, 
                                             mode.lower(), 
                                             data_quality)

## Plot new-vs-cumulative (all and last 3 months)

In [11]:
if True: # switch this output on/off

    THREE_MONTHS = 93 # days
    periods = [0, THREE_MONTHS]

    tags = ["full", "3months"]  
    for period, tag in zip(periods, tags):
        for mode, index in modes.items():
            for name in adj_cum_data[index].columns:
        
                # let's not plot empty charts
                if adj_daily_data[index][name][-period:].sum() == 0: 
                    continue
            
                ps.plot_new_cum(
                    adj_daily_data[index][name][-period:].copy(), 
                    adj_cum_data[index][name][-period:].copy(), 
                    mode, name, 
                    title=f'{name}: COVID-19 {mode.title()}',
                    rfooter=data_quality[index][name],
                    save_as=f'{CHART_DIRECTORY}{name}-{mode.lower()}'
                                f'-new-vs-cum-{tag}.png'
                )

## International comparisons - maps/leader-boards/swarms

In [12]:
def per_million_population(population: pd.Series, power:int = 6):
    """Take a population series and a power and return in a tuple:
       - the power
       - the factor (which is 10 ** power)
       - an updated population series (which is population / factor)"""
    
    factor = 10 ** power
    return power, factor, population / factor

In [13]:
def get_rear_offsets(dataframe, period):
    """Get offset to last non-zero values in dataframe by col
       provided offset is within period, otherwise zero offset
       [We do this becuase some nations are slow in reporting,
       and without this adjustment late reporting nations would 
       look better than they actually are.]"""
    
    nrows = len(dataframe)
    rear_offsets = pd.Series(0, index=dataframe.columns)
    for col in dataframe.columns:
        index_array = (np.nonzero(dataframe[col].to_numpy()))[0]
        if len(index_array) > 0:
            last = index_array[-1]
            rear_offsets[col] = nrows - last - 1
            # Note: rear_offsets[col] is 0 if len(index_array) == 0
    rear_offsets = rear_offsets.where(rear_offsets<=period, other=0)
    return rear_offsets

def get_recent_total(dataframe, rear_offsets, period):
    """Sum the last rows of a dataframe, making adjustments
       for zero rows at the very end"""
    
    daily_sum = pd.Series(0.0, index=dataframe.columns)
    for col in dataframe.columns:
        p = 0 - (period+rear_offsets[col])
        daily_sum[col] = dataframe[col].iloc[p:].sum()
    return daily_sum

def get_larger_nations(population, thresh=100_000):
    """return a list of nations with a population exceeding thresh"""
    
    return population[population >= thresh].index

In [14]:
def map_world(series, title, legend_title, source):
    
    # prepare data for mapping
    name_map(series, iso_name_map)
    score = pd.DataFrame(series) # back to DataFrame
    score.columns = ['Score']
    score['country'] = score.index
    score['code'] = [iso3166.countries.get(x.upper())[2] 
                     for x in score['country']]
    
    # get map data
    shapefile = ('../geo-data/ne_110m_admin_0_countries/'
        'ne_110m_admin_0_countries.shp')
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf = gdf[gdf['country'] != 'Antarctica'] 
    
    merged = gdf.merge(score,
                       left_on='country_code', 
                       right_on='code', how='left')

    variable = 'Score'
    cmap = mpl.cm.get_cmap('viridis').reversed()
    cmap.set_bad('white')
    cmap.set_under('white')
    ax = merged.plot(column=variable, cmap=cmap, legend=False)
    
    # colorbar
    world_map = ax.collections[0]
    cb = plt.colorbar(world_map, ax=ax, orientation='horizontal')
    
    # legend title
    fig = ax.figure
    fig.text(0.5, 0.175, legend_title,
        ha='center', va='bottom',
        fontsize=12, # fontstyle='italic',
        color='#222222')
    
    ps.finalise_plot(ax, title=title,
                     xticklabels=[], yticklabels=[],
                     xticks=[], yticks=[],
                     rfooter=source,
                     set_size_inches=(8,5),
                     save_as=f'{CHART_DIRECTORY}'
                             f'{I_PREFIX}MAP-{title}.png',
                    )

In [15]:
def swarm(data: pd.Series,
          title="Don't forget the title",
          ylabel="Don't forget the ylabel",
          source="Don't forget the source",
          color='cornflowerblue'):
    """Produce a swarm plot from the following input:
        - data - a pandas Series of values, with an index of nations
        - title - plot title
        - ylabel - label for the y axis
        - source - string for data source - becomes right footer
        - color - colour of swarm plot dots
       """
    
    # get country information
    wbd = pd.read_excel('../data/CLASS.xls', header=4, index_col=0, 
                        ).iloc[1:219].dropna(how='all', axis=1)
    mapping = wbd['Income group']
    mapping.index = wbd['Code']
    
    # prepare for plot
    name_map(data, iso_name_map)
    data = pd.DataFrame(data)
    data.columns = ['Rate']
    data['alpha3'] = [iso3166.countries.get(x).alpha3 for x in data.index]
    data['alpha2'] = [iso3166.countries.get(x).alpha2 for x in data.index]
    data['Income Group'] = data.alpha3.map(mapping)
    data = data[data['Income Group'].notna()]
    
    # labels 
    data['alpha2'] = data['alpha2'].where(data['alpha2'].notna(), other='')
    
    # swarm plot
    # - set up
    categories = ['Low income', 'Lower middle income',
                  'Upper middle income', 'High income']
    # - plot
    fig, ax = plt.subplots(figsize=(8,5))
    sns.swarmplot(x='Income Group', y='Rate', data=data, 
                       size=10,
                       dodge=True, 
                       color=color, alpha=0.5,
                       order=categories,
                       ax=ax)
    
    # === Next code broken with Matplotlib v3.3.3 ===
    # - point annotation - this is one ugly hack
    #for collect, name in zip(ax.collections, categories):
    #    
    #    # retrieve positional data from plot
    #    op = collect.get_offset_position()
    #    collect.set_offset_position('data')
    #    retrieved_xy_pairs = collect.get_offsets()
    #    collect.set_offset_position(op)
    #    
    #    # build an xy map with duplicate keys
    #    THRESH = 7
    #    dup_map = {}
    #    for x, y in retrieved_xy_pairs:
    #        yy = np.round(y, THRESH)
    #        if yy in dup_map:
    #            dup_map[yy].append(x)
    #        else:
    #            dup_map[yy] = [x]
    #    
    #    # use this map to plot in the data labels
    #   for index, row in data[data['Income Group'] == name].iterrows():
    #        lookup = np.round(row['Rate'], THRESH)
    #        if lookup in dup_map:
    #           ax.text(dup_map[lookup].pop(), row['Rate'], row['alpha2'],
    #                   ha='center', va='center', fontsize='xx-small', 
    #                    color='#333333')
        
    ps.finalise_plot(ax, title=title,
                     xlabel=None, ylabel=ylabel,
                     rfooter=source,
                     set_size_inches=(8, 8),
                     chart_directory=CHART_DIRECTORY
                    )

In [16]:
if True: # switch this output on/off

    # set-up
    PERIOD = 7 # days (recent data period for daily averages)
    THRESH = 100_000 # people (minimum nation size for plotting)
    BAR_N = 40 # maximum bars on chart
    BAR_PLOT_SIZE = (8, 8)
    keepers = get_larger_nations(population, THRESH)
    power, factor, pop_millions =  per_million_population(population)

    # for cases then deaths ...
    for mode, index in modes.items():
        print(mode)
    
        # collect the data for plotting
        raw_dataframe = adj_daily_data[index].copy()
        
        # drop World
        if 'World' in raw_dataframe.columns:
            del raw_dataframe['World']
        if 'World' in pop_millions.index:
            pop_millions.drop(labels='World', inplace=True)
            keepers = keepers.drop(labels='World')
    
        # - cumulative data
        cumulative = raw_dataframe.sum()
        cumulative_percapita = (cumulative / pop_millions)[keepers]
        log_cumulative_percapita = np.log(cumulative_percapita + 1)
        # - latest daily average data
        rear_offsets = get_rear_offsets(raw_dataframe, PERIOD)
        daily_ave = (get_recent_total(raw_dataframe, rear_offsets, 
                                      PERIOD) / PERIOD)
        daily_ave_percapita = (daily_ave / pop_millions)[keepers]
        log_daily_ave_percapita = np.log(daily_ave_percapita + 1)

        # bar charts
        # - bar chart of the top cumulative performers
        top_tier = cumulative.sort_values(ascending=True)[-BAR_N:]
        ps.plot_barh(
            series = np.round(top_tier.copy(), 0).astype(int), 
            title=(f'COVID-19: Top cumulative {mode.lower()}'),
            xlabel=(f'Cumulative {mode.lower()}'),
            lfooter=f'For nations with a population >= {THRESH:,}',
            rfooter=f'Source: {source}',
            set_size_inches=BAR_PLOT_SIZE,
            save_as=f'{CHART_DIRECTORY}{I_PREFIX}Top-{mode.title()} (cum).png'
        )

        # - bar chart of the top cumulative performers per capita
        top_tier = cumulative_percapita.sort_values(ascending=True)[-BAR_N:]
        ps.plot_barh(
            series = np.round(top_tier.copy(), 1), 
            title=(f'COVID-19: Top cumulative {mode.lower()} per capita'),
            xlabel=(f'Cumulative {mode.lower()} per '
                    f'$10^{power}$ population'),
            lfooter=f'For nations with a population >= {THRESH:,}',
            rfooter=f'Source: {source}',
            set_size_inches=BAR_PLOT_SIZE,
            save_as=(f'{CHART_DIRECTORY}{I_PREFIX}TOP-{mode.title()}'
                     ' per capita (cum).png')
        )

        # - bar chart of the top daily averages - past week
        top_tier = daily_ave.sort_values(ascending=True)[-BAR_N:]
        ps.plot_barh(
            series = np.round(top_tier.copy(), 1), 
            title=(f'COVID-19: Top {mode.lower()} - '
                  f'past {PERIOD} days'),
            xlabel=(f'Average daily {mode.lower()}'),
            lfooter=f'For nations with a population >= {THRESH:,}',
            rfooter=f'Source: {source}',
            set_size_inches=BAR_PLOT_SIZE,
            save_as=(f'{CHART_DIRECTORY}{I_PREFIX}Top-{mode.title()} '
                     '(recent).png')
        ) 
        
        # - bar chart of the top daily averages per capita
        top_tier = daily_ave_percapita.sort_values(ascending=True)[-BAR_N:]
        ps.plot_barh(
            series = np.round(top_tier.copy(), 1), 
            title=(f'COVID-19: Top {mode.lower()} per capita - '
                  f'past {PERIOD} days'),
            xlabel=(f'Average daily {mode.lower()} per '
                    f'$10^{power}$ population'),
            lfooter=f'For nations with a population >= {THRESH:,}',
            rfooter=f'Source: {source}',
            set_size_inches=BAR_PLOT_SIZE,
            save_as=(f'{CHART_DIRECTORY}{I_PREFIX}Top-'
                     f'{mode.title()} per capita (recent).png')
        )  

        # world maps
        # - world map - cumulative per capita
        title = f'COVID-19 Cumulative {mode.lower()} per capita'
        legend = (f'Cumulative {mode.lower()} per '
                  f'$10^{power}$ population')
        map_world(cumulative_percapita.copy(), title, legend,
                  f'Source: {source}')

        # - world map - cumulative per capita - log scale
        title = (f'COVID-19 Cumulative {mode.lower()} per capita '
                 f'(log scale)')
        legend = (f'log((cumulative {mode.lower()} per '
                  f'$10^{power}$ pop.) + 1)')
        map_world(log_cumulative_percapita.copy(), title, legend,
                  f'Source: {source}')
        
        # - world map - average daily per capita past PERIOD days
        title = (f'COVID-19 Ave. daily {mode.lower()} per capita '
             f'- past {PERIOD} days')
        legend = (f'Ave. daily {mode.lower()} per '
                  f'$10^{power}$ population')
        map_world(daily_ave_percapita.copy(), title, legend, 
                  f'Source: {source}')

        # - world map - average daily per capita past PERIOD days - log scale
        title = (f'COVID-19 Ave. daily {mode.lower()} per capita '
             f'- past {PERIOD} days (log scale)')
        legend = (f'log((average daily {mode.lower()} per '
              f'$10^{power}$-population) + 1)')
        map_world(log_daily_ave_percapita.copy(), title, legend,
                  f'Source: {source}')

        # === SWARM plots are broken - need to fix ===
        
        # swarm plots
        ## - swarm plot of cumulative per capita
        #title = f'COVID-19 Cumulative {mode.lower()} per capita'
        #ylabel = (f'Cumulative {mode.lower()} per '
        #          f'$10^{power}$ population')
        #swarm(cumulative_percapita.copy(), title, 
        #      ylabel, f'Source: {source}', color='darkorchid')        
        #
        ## - swarm plot of cumulative per capita - log scale
        #log_cumulative_percapita = np.log(cumulative_percapita + 1)
        #title = f'COVID-19 Cumulative {mode.lower()} per capita (log scale)'
        #ylabel = (f'log((cumulative {mode.lower()} per '
        #          f'$10^{power}$ pop.) + 1)')
        #swarm(log_cumulative_percapita.copy(), title, 
        #      ylabel, f'Source: {source}', color='hotpink')
        #
        ## - swarm plot of daily average per capita past PERIOD days
        #title = (f'COVID-19 Ave. daily {mode.lower()} per capita '
        #     f'- past {PERIOD} days')
        #ylabel = (f'Ave. daily {mode.lower()} per '
        #         f'$10^{power}$ population')
        #swarm(daily_ave_percapita.copy(), title, 
        #      ylabel, f'Source: {source}')
        #
        ## - swarm plot of daily average per capita past PERIOD days - log scale
        #title = (f'COVID-19 Ave. daily {mode.lower()} per capita '
        #     f'- past {PERIOD} days (log scale)')
        #ylabel = (f'log((ave. daily {mode.lower()} per '
        #         f'$10^{power}$ pop.) + 1)')
        #swarm(log_daily_ave_percapita.copy(), title, 
        #      ylabel, f'Source: {source}', color='darkorange')        

Cases
Deaths


## Semi-log comparison plots of cumulative

In [17]:
def plot_semi_log_trajectory(data, mode, threshold, selections, source):
    """Produce semi-log plots of cumulative data from the 
       following inputs:
        - data - a pandas dataframe of cumulative data
        - mode - a string, either 'Cases' or 'Deaths'
        - threshold - starting point (eg. 100th case)
        - selections - a python dictionary of 
         'grouping': ['list', 'of', 'nations'], pairs
        - source - string for data source
       """
    
    styles = ['-'] #, '--', '-.', ':'] # 4 lines 
    markers = list('PXo^v<>D*pH.d') # 13 markers
    colours = ['blue', 'red', 'maroon', 'darkorange', 'brown', 
               'olivedrab', 'darkgoldenrod', 'green',  
               'purple', 'black', 'teal'] # 11 colours

    for tag, nation_list in selections.items():
    
        # set up for the plot
        ax = plt.subplot(111,)
        ax.set_title('COVID-19 Semilog plot of selected '
                     f'{mode.lower()[:-1]} trajectories')
        ax.set_xlabel('Days from the notional '
                      f'{int(threshold)}th {mode.lower()[:-1]}')
        ax.set_ylabel(f'Cumulative {mode} (log scale)')
        ax.set_yscale('log')

        # add the data
        endpoints = pd.DataFrame()
        nation_list.sort()
        for i, name in enumerate(nation_list):
            if name not in data.columns:
                print(f'{name} is not in data')
                continue
            y = data[name]
            y.index = range(len(y))
            start = y[y >= threshold]
            if len(start) < 5:
                continue
            start = start.index[0] - 1
            if start < 0:
                start = 0
            y = y[start:].values
            x = range(-1, len(y) - 1)
            
            # plot line
            code = get_national_code(name)
            label = f'{name} ({code}) {int(y[-1]):,}'
            color = colours[i % len(colours)]
            ax.plot(x, y, label=label,
                    color=color,)
            
            # plot end text
            endpoints = endpoints.append(
                pd.Series([x[-1], y[-1], f'{code}', color],
                          index=['x', 'y', 'code', 'color'],
                          name=name))

        # add endpoint labels 
        EXPANSION = 0.02
        additional = endpoints.x.max() * EXPANSION
        for row in endpoints.itertuples(): 
            ax.text(x=row.x+additional, y=row.y, s=row.code,
                    size='small', color='black',
                    ha='left', va='center',
                    bbox={'alpha':0.5, 'facecolor':'white'})

        # etc.
        min_, max_ = ax.get_xlim()
        max_ = max_ + (max_ * EXPANSION)
        ax.set_xlim(min_, max_)
        ax.legend(loc='lower right', fontsize='8', ncol=3)
        fig = ax.figure

        # footnote the plot
        fig.text(0.99, 0.01, source,
            ha='right', va='bottom',
            fontsize=9, fontstyle='italic',
            color='#999999')

        # finalise 
        fig.set_size_inches(8, 6)
        fig.tight_layout(pad=1.2)
        fig.savefig(f'{CHART_DIRECTORY}{I_PREFIX}semilog-{mode}-{tag}', dpi=125)
        #plt.show()
        plt.close()

In [18]:
if True: # switch this output on/off

    selections = {
        'Anglophone':  ['Australia', 'United States', 'Canada', 
                        'United Kingdom', 'New Zealand', 'Ireland', ],
        'Neighbours':  ['Australia', 'New Zealand', 'Papua New Guinea',
                        'Indonesia', 'Singapore', 'Malaysia', #'Brunei',
                        'Timor', 'China', 'Japan', 'South Korea', 'India'],
        "SelectedEurope":['Austria', 'Belgium', 'Denmark', 
                        'France', 'Germany', 'Greece', 'Italy', 'Netherlands', 
                        'Norway', 'Poland', 'Portugal', 'Russia', 'Spain',  
                        'Sweden', 'Switzerland', 'United Kingdom', ],
        "EU-v-US":     ['United States', "European Union"],
        "Italy-v-UK":  ['Italy', "United Kingdom"]
    }
    
    # https://europa.eu/european-union/about-eu/countries_en
    EU = [
        'Austria', 'Belgium', 'Bulgaria', 'Croatia', 
        'Cyprus', 'Czechia', 'Denmark',
        'Estonia', 'Finland', 'France', 'Germany',
        'Greece', 'Hungary', 'Ireland', 'Italy',
        'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 
        'Netherlands', 'Poland', 'Portugal', 'Romania', 
        'Slovakia', 'Slovenia', 'Spain', 'Sweden', 
    ]
    assert(len(EU) == 27) 
    
    # for cases then deaths ...
    for mode, index in modes.items():
        print(mode)
        cumulative = adj_cum_data[index].copy()
        if 'World' in cumulative.columns:
            del cumulative['World']
        cumulative["European Union"] = cumulative[EU].sum(axis=1)
        
        # include top N nations as a chart
        N = 9
        top = cumulative.iloc[-1].sort_values(ascending=False)[:N]
        selections['Top'] = top.index.to_list()
        
        # and plot
        thresh = 100 if mode.lower() == 'cases' else 10
        plot_semi_log_trajectory(cumulative, mode, thresh, 
                                 selections, f'Source: {source}')

Cases
Deaths
Timor is not in data


## Comparison plots of daily new per capita

In [19]:
def plot_daily_comparative(data, population, mode, regions, source, period=14):
    """Produce daily plots highlighting new cases/deaths for a 
       specific set of nations against a backdrop of all nations.
    Inputs:
        - data - pandas DataFrame of daily new cases/deaths
        - population - pandas Series of national populations 
        - mode - string - either "cases" or "deaths"
        - regions - python list of lists of strings 
          (nation names - same as columns in data)
        - source - string for data source
        - period - period for the rolling average """
    
    # set-up
    colours = ['blue', 'red', 'maroon', 'darkorange', 'brown', 
               'olivedrab', 'darkgoldenrod', 'green',  
               'purple', 'black', 'teal'] # 11 colours
    power, factor, pop_millions =  per_million_population(population)
    keepers = get_larger_nations(population, 100_000) # used for backgrounds

    # get rolling average per capita from start date
    df = data.rolling(period).mean().div(other=pop_millions)
    df = df[df.index >= pd.Timestamp('2020-02-01')]

    for region_list in regions:
    
        # plot background
        ax = df[keepers].plot(c='#aaaaaa', lw=0.5)
        ax.get_legend().remove()

        # plot highlighted regions
        region_list.sort()
        ax_new = ax.twinx()        
        for i, name in enumerate(region_list):
            df[name].plot(c=colours[i], label=name, lw=2.5, ax=ax_new)
        ax_new.legend(title=None, loc="upper left")
        ax_new.grid(False)
        ax_new.set_yticklabels([])
        ax_new.set_ylim(ax.get_ylim())

        ps.finalise_plot(ax,
                      title=f'COVID-19 Daily New {mode.title()}',
                      xlabel=None,
                      ylabel=f'Daily new {mode.lower()} per $10^{power}$ '
                              f'population\n{period}-day rolling average',
                      rfooter=source,
                      set_size_inches=(8, 5),
                      save_as=f'{CHART_DIRECTORY}{I_PREFIX}'
                              f'daily-{mode}-{" ".join(region_list)}',
                     )


In [20]:
if True: # switch this output on/off

    # identify the regional sets to be plotted
    regions = [
        ['European Union', 'United States'],
        ['Belgium', 'Ireland', 'Netherlands', 'United Kingdom', 'Iceland'],
        ['France', 'Italy', 'Portugal', 'Spain'],
        ['Denmark', 'Norway', 'Sweden', 'Austria', 'Germany', 'Switzerland'], 
        ['Finland', 'Estonia', 'Latvia', 'Lithuania', 'Poland'],
        ['Belarus', 'Russia', 'Ukraine', 'Romania', 'Bulgaria', 'Moldova', ],
        ['Czechia', 'Slovakia', 'Hungary', 'Slovenia'],
        ['Croatia', 'Bosnia and Herzegovina', 'Montenegro',
            'Serbia', 'Kosovo', 'Albania', 'North Macedonia', 'Greece'],
        ['Turkey', 'Syria', 'Lebanon', 'Israel', 'Jordan', 
             'Egypt', 'Libya', 'Cyprus'],
        ['Georgia', 'Armenia', 'Azerbaijan'],
        ['Iraq', 'Iran', 'Saudi Arabia', 'Bahrain', 'Qatar', 
            'United Arab Emirates', 'Kuwait', 'Yemen', 'Oman',], 
        ['Ghana', 'Ethiopia', 'Kenya', 
             'Nigeria', 'South Africa', 'Tanzania'],
        ['Afghanistan', 'Kazakhstan', 'Kyrgyzstan', 'Pakistan', 
            'Tajikistan', 'Uzbekistan', ],
        ['Bangladesh', 'Nepal', 'India', 'Sri Lanka'], 
        ['Indonesia', 'Malaysia', 'Philippines', 'Singapore', 'Thailand'],
        ['China', 'Japan', 'South Korea', 'Taiwan', 'Vietnam'],
        ['Australia', 'New Zealand', 'Papua New Guinea', 'Timor'], 
        ['Canada', 'Mexico', 'United States'], 
        ['Costa Rica', 'Guatemala', 'Honduras', 'Nicaragua', 'Panama'], 
        ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 
            'Peru', 'Venezuela'], 
        ['Cuba', 'Dominican Republic', 'Haiti', 'Jamaica'],
    ]

    # And population for European Union
    pop = population.copy()
    pop['European Union'] = pop[EU].sum()
    # FYI only ...
    print('Populations:\n'
          f'United States: {int(pop["United States"]):,}\n'
          f'European Union: {int(pop["European Union"]):,}\n')

    # for cases then deaths ...
    for mode, index in modes.items():
        print(mode)
        daily = adj_daily_data[index].copy()
        daily['European Union'] = daily[EU].sum(axis=1)
        plot_daily_comparative(daily, pop, mode, regions, f'Source: {source}')

Populations:
United States: 331,002,647
European Union: 444,919,060

Cases
Deaths


## Growth

In [21]:
def plot_growth_factor(data, mode, source): 
    #data, _ = limit_data_cols_to_threshold(data, mode)

    for name in data.columns:
        ps.plot_growth_factor(data[name], 
            title=f'{name}: w/w growth in new COVID-19 {mode.lower()}',
            ylabel='Growth factor',
            xlabel=None,
            save_as=f'{CHART_DIRECTORY}/{name}-{mode}-growth-factor-{source}.png',
            rfooter=source,
            lfooter=f'Weekly rolling average daily new {mode.lower()} this week / last week'
        )

In [22]:
if True: # switch this output on/off
    # for cases then deaths ...
    for mode, index in modes.items():
        daily = adj_daily_data[index].copy()
        plot_growth_factor(daily, mode, source)

## The End

In [23]:
print('Finished')

Finished
